# Progetto - Complementi di Basi di Dati A.A. 2023/2024

## Consegna Progetto

#### Introduzione

L'obiettivo del progetto è condurre un'analisi dettagliata di un DB a scelta, da confrontare con i DBMS visti a lezione (i.e., Mysql, MongoDB, Neo4J, Elasticsearch).

Un'analisi approfondita del DB è fondamentale per comprenderne le caratteristiche, identificare eventuali problemi e suggerire soluzioni per migliorare le prestazioni complessive.

Il progetto può essere svolto in due modalità:

1. Creando un file *docker-compose.yml* con all'interno l'indicazione di un servizio relativo al DBMS che avete scelto;
2. Installare il DBMS sulla vostra macchina.

Non essendo **Docker** un argomento del corso, non è obbligatorio il suo utilizzo per lo svolgimento del progetto. Per chi fosse interessato, sulla pagina del corso mettiamo a disposizione il materiale dei laboratori come esempio di utilizzo di Docker (Ricordiamo che è possibile cercare le immagini Docker all'interno di [Docker Hub](https://hub.docker.com/)).

La connessione e le query verranno svolte utilizzando il linguaggio **Python** all'interno di questo file **Jupyter**.

Il progetto deve affrontare i seguenti:

1. Introduzione e Descrizione del DBMS scelto;
2. Libreria/e DBMS;
3. Creazione e Connessione al DB;
4. Operazioni CRUD;
5. Comparazione con i DBMS visti a lezione;
6. Esempi di query;
7. Analisi delle tempistiche delle query.

#### Consegna

E'richiesto l'invio del file Jupyter.

Per le celle contenenti il codice, l'output dovrà essere presente nel file di consegna.

#### Nota

Di seguito riportiamo le indicazioni da seguire per lo svolgimento del progetto.

### Introduzione e descrizione del DBMS scelto

Questa sezione ha lo scopo di descrivere il DBMS scelto per lo svolgimento del progetto:

- Descrizione del DBMS;
- Caratteristiche del DBMS;
- Installazione su Windows, MacOS e Linux.
- Pro e Contro;
- Analisi di servizi/prodotti che utilizzano il DBMS in analisi.

### Libreria/e DBMS

Descrivere la libreria o le librerie necessarie per connettersi al DBMS scelto (analogamente a **pymongo** per MongoDB e **py2neo** per Neo4J):
- Caratteristiche della libreria (e.g., nome, modalità di installazione);
- Indicazione del link alla documentazione.



In [1]:
pip install crate

Note: you may need to restart the kernel to use updated packages.


### Creazione e connessione al DB

Mostrare come creare un'istanza del DBMS e come effettuare la connessione al DB in modo da gestire i dati (utilizzando le librerie installate e descritte sopra).

L'istanza creata dovrà essere utilizzata per gestire i dati nelle prossime sezioni.

In [2]:
from crate import client

with client.connect("http://cratedb_container:4200", username="crate") as connection:
    print("connected")

connected


In [3]:
import json
from crate import client
import traceback

# Define the function to load data from a JSON file
def load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(traceback.format_exc())
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        print(traceback.format_exc())
        return None

# Define the SQL query to create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS drugs_external (
  "DrugBank ID" TEXT PRIMARY KEY,
  "Name" TEXT,
  "CAS Number" TEXT,
  "Drug Type" TEXT,
  "UniProt ID" TEXT,
  "UniProt Title" TEXT,
  "Drugs com Link" TEXT,
  "BindingDB ID" INT
);
"""

# Define the SQL query to insert data
insert_query = """
INSERT INTO drugs_external ("DrugBank ID", "Name", "CAS Number", "Drug Type", "UniProt ID", "UniProt Title", "Drugs com Link", "BindingDB ID")
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""


# Load data from the JSON file
data = load_json('externaldrugjson.json')
if data is None:
    print("Failed to load JSON data. Exiting.")
    exit(1)

# Connect to the CrateDB server
try:
    connection = client.connect("http://cratedb_container:4200", username="crate")
except Exception as e:
    print(f"Error connecting to CrateDB: {e}")
    print(traceback.format_exc())
    exit(1)

# Create a cursor object
cursor = connection.cursor()

# Create the table if it doesn't exist
try:
    cursor.execute(create_table_query)
    connection.commit()
except Exception as e:
    print(f"Error creating table: {e}")
    print(traceback.format_exc())
    cursor.close()
    connection.close()
    exit(1)

# Insert each record from the JSON data
try:
    for record in data:
        # Check if BindingDB ID is an empty string
        bindingdb_id = record['BindingDB ID']
        if not bindingdb_id:
            bindingdb_id = None 

        cursor.execute(insert_query, (
            record['DrugBank ID'],
            record['Name'],  
            record['CAS Number'],
            record['Drug Type'],
            record['UniProt ID'],
            record['UniProt Title'],
            record['Drugs.com Link'],
            bindingdb_id  
        ))
    # Commit the transaction
    connection.commit()
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())




### Operazioni CRUD

Mostrare come vengono affrontate le operazioni CRUD nel DB in analisi, evidenziando sia la sintassi, sia esempi concreti per ogni operazione:

1. **Create**: Aggiunta dei dati sia in modalità singola che multipla. Quindi come viene aggiunto un singolo dato e un insieme di dati al DB (analogo alla *insert_one* e *insert_many* di MongoDB);
2. **Read**: Lettura dei dati dal DB. In questo caso devono essere mostrati i metodi base per la lettura dei dati:
    - Lettura di tutti i dati;
    - Una semplice query di selezione a vostro piacere.
3. **Update**: Aggiornamento dei dati presenti nel database. Mostrare due query che aggiornano i dati inseriti precedentemente;
4. **Delete**: Eliminazione dei dati sia in modalità singola che multipla. Mostrare una eliminazione singola e una multipla.

## CREATE

### Insert one record

In [4]:
import time

insert_query = """
INSERT INTO drugs_external ("DrugBank ID", "Name", "CAS Number", "Drug Type", "UniProt ID", "UniProt Title", "Drugs com Link", "BindingDB ID")
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""

new_record = {
    "DrugBank ID": "DB123456",
    "Name": "New Drug",
    "CAS Number": "123-45-6",
    "Drug Type": "SmallMoleculeDrug",
    "UniProt ID": "U12345",
    "UniProt Title": "New Protein",
    "Drugs com Link": "https://example.com",
    "BindingDB ID": 789
}

start_time = time.time()
try:
    cursor.execute(insert_query, (
        new_record["DrugBank ID"],
        new_record["Name"],
        new_record["CAS Number"],
        new_record["Drug Type"],
        new_record["UniProt ID"],
        new_record["UniProt Title"],
        new_record["Drugs com Link"],
        new_record["BindingDB ID"]
    ))
    connection.commit()
    print("Record inserted successfully.")
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to insert: {execution_time} seconds")

Record inserted successfully.
Time taken to insert: 0.0024743080139160156 seconds


### Insert multiple records

In [5]:
insert_query = """
INSERT INTO drugs_external ("DrugBank ID", "Name", "CAS Number", "Drug Type", "UniProt ID", "UniProt Title", "Drugs com Link", "BindingDB ID")
VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""

records_to_insert = [
    ("DB123457", "Drug1", "123-45-7", "SmallMoleculeDrug", "U123456", "Protein1", "https://example.com", 789),
    ("DB123458", "Drug2", "123-45-8", "BiotechDrug", "U123457", "Protein2", "https://example.com", 790),
    ("DB123459", "Drug3", "123-45-9", "BiotechDrug", "U123458", "Protein3", "https://example.com", 791),
    ("DB123460", "Drug4", "123-45-10", "BiotechDrug", "U123459", "Protein4", "https://example.com", 792),
    ("DB123461", "Drug5", "123-45-11", "SmallMoleculeDrug", "U123460", "Protein5", "https://example.com", 793)
]

start_time = time.time()
try:
    cursor.executemany(insert_query, records_to_insert)
    # Commit the transaction
    connection.commit()
    print("Records inserted successfully.")
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to insert 5 records: {execution_time} seconds")

Records inserted successfully.
Time taken to insert 5 records: 0.002200603485107422 seconds


## READ

### Fetch all data

In [6]:
select_query = """
SELECT * FROM drugs_external;
"""

start_time = time.time()
try:
    cursor.execute(select_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time taken to fetch: {execution_time} seconds")
except Exception as e:
    print(f"Error executing SELECT query: {e}")
    print(traceback.format_exc())

['DB00001', 'Lepirudin', '138068-37-8', 'BiotechDrug', 'P01050', 'ITH1_HIRME', 'http://www.drugs.com/cdi/lepirudin.html', None]
['DB00012', 'Darbepoetin alfa', '209810-58-2', 'BiotechDrug', 'P01588', 'EPO_HUMAN', 'http://www.drugs.com/cdi/darbepoetin-alfa-albumin.html', None]
['DB00019', 'Pegfilgrastim', '208265-92-3', 'BiotechDrug', 'P09919', 'CSF3_HUMAN', 'http://www.drugs.com/cdi/pegfilgrastim.html', None]
['DB00020', 'Sargramostim', '123774-72-1', 'BiotechDrug', 'P04141', 'CSF2_HUMAN', 'http://www.drugs.com/cdi/sargramostim.html', None]
['DB00025', 'Antihemophilic factor, human recombinant', '139076-62-3', 'BiotechDrug', 'P00451', 'FA8_HUMAN', 'http://www.drugs.com/cdi/antihemophilic-factor-human.html', None]
['DB00031', 'Tenecteplase', '191588-94-0', 'BiotechDrug', 'P00750', 'TPA_HUMAN', 'http://www.drugs.com/mtm/tenecteplase.html', None]
['DB00036', 'Coagulation factor VIIa Recombinant Human', '102786-61-8', 'BiotechDrug', 'P08709', 'FA7_HUMAN', 'http://www.drugs.com/cdi/coagulat

### SELECT DrugBank ID and Name of the drugs that have BindingDB ID not null

In [7]:
select_query = """
SELECT "DrugBank ID", "Name" FROM drugs_external WHERE "BindingDB ID" IS NOT NULL;
"""

start_time = time.time()
try:
    cursor.execute(select_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing SELECT query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute SELECT query: {execution_time} seconds")

['DB00050', 'Cetrorelix']
['DB00104', 'Octreotide']
['DB00123', 'Lysine']
['DB00134', 'Methionine']
['DB00136', 'Calcitriol']
['DB00141', 'N-Acetylglucosamine']
['DB00143', 'Glutathione']
['DB00147', 'Pyridoxal']
['DB00151', 'Cysteine']
['DB00154', 'Dihomo-gamma-linolenic acid']
['DB00159', 'Icosapent']
['DB00162', 'Vitamin A']
['DB00166', 'Lipoic acid']
['DB00167', 'Isoleucine']
['DB00171', 'ATP']
['DB00174', 'Asparagine']
['DB00175', 'Pravastatin']
['DB00176', 'Fluvoxamine']
['DB00187', 'Esmolol']
['DB00218', 'Moxifloxacin']
['DB00222', 'Glimepiride']
['DB00224', 'Indinavir']
['DB00229', 'Cefotiam']
['DB00233', 'Aminosalicylic acid']
['DB00234', 'Reboxetine']
['DB00235', 'Milrinone']
['DB00245', 'Benzatropine']
['DB00247', 'Methysergide']
['DB00268', 'Ropinirole']
['DB00277', 'Theophylline']
['DB00283', 'Clemastine']
['DB00287', 'Travoprost']
['DB00290', 'Bleomycin']
['DB00291', 'Chlorambucil']
['DB00319', 'Piperacillin']
['DB00320', 'Dihydroergotamine']
['DB00321', 'Amitriptyline']


## UPDATE

### UPDATE the record ("DB123457", "Drug1", "123-45-7", "SmallMoleculeDrug", "U123456", "Protein1", "https://example.com", 789) and set new values ("DB123457", "Drug15", "420-45-7", "SmallMoleculeDrug", "U654321", "Protein15", "https://example.com/protein")

In [8]:
update_query = """
UPDATE drugs_external 
SET "Name" = ?, "CAS Number" = ?, "Drug Type" = ?, "UniProt ID" = ?, "UniProt Title" = ?, "Drugs com Link" = ?, "BindingDB ID" = ?
WHERE "DrugBank ID" = ?;
"""

new_values = (
    "Drug15", 
    "420-45-7", 
    "SmallMoleculeDrug", 
    "U654321", 
    "Protein15", 
    "https://example.com/protein", 
    789, 
    "DB123457"
)

start_time = time.time()
try:
    cursor.execute(update_query, new_values)
    connection.commit()
    print("Record updated successfully.")
except Exception as e:
    print(f"Error updating data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to update: {execution_time} seconds")

Record updated successfully.
Time taken to update: 0.005795001983642578 seconds


### UPDATE the record ("DB123460", "Drug4", "123-45-10", "SmallMoleculeDrug", "U123459", "Protein4", "https://example.com", 792) and set new values ("DB123460", "Drug45", "321-45-10", "ANewKindOfDrug", "U658428", "Protein45", "https://example.com/newdrug", 792)

In [9]:
update_query = """
UPDATE drugs_external 
SET "Name" = ?, "CAS Number" = ?, "Drug Type" = ?, "UniProt ID" = ?, "UniProt Title" = ?, "Drugs com Link" = ?, "BindingDB ID" = ?
WHERE "DrugBank ID" = ?;
"""

new_values = (
    "Drug45", 
    "321-45-10", 
    "ANewKindOfDrug", 
    "U658428", 
    "Protein45", 
    "https://example.com/newdrug", 
    792, 
    "DB123460"
)

start_time = time.time()
try:
    cursor.execute(update_query, new_values)
    connection.commit()
    print("Record updated successfully.")
except Exception as e:
    print(f"Error updating data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to update: {execution_time} seconds")

Record updated successfully.
Time taken to update: 0.0032248497009277344 seconds


## DELETE

### DELETE ONE RECORD

In [10]:
delete_query = """
DELETE FROM drugs_external WHERE "DrugBank ID" = ?;
"""

drugbank_id_to_delete = "DB123456"

start_time = time.time()
try:
    cursor.execute(delete_query, (drugbank_id_to_delete,))
    connection.commit()
    print("Record deleted successfully.")
except Exception as e:
    print(f"Error deleting data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to delete: {execution_time} seconds")



Record deleted successfully.
Time taken to delete: 0.001916646957397461 seconds


### DELETE MULTIPLE RECORDS

In [11]:
delete_query = """
DELETE FROM drugs_external WHERE "DrugBank ID" IN (?, ?, ?, ?, ?);
"""

drugbank_ids_to_delete = ("DB123457", "DB123458", "DB123459", "DB123460", "DB123461")

start_time = time.time()
try:
    cursor.execute(delete_query, drugbank_ids_to_delete)
    connection.commit()
    print("Records deleted successfully.")
except Exception as e:
    print(f"Error deleting data: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to delete 5 records: {execution_time} seconds")

Records deleted successfully.
Time taken to delete 5 records: 0.0028541088104248047 seconds


## Comparazione con i DBMS visti a lezione

Mostrare le principali differenze con i DB visti a lezione:

- Differenze rispetto a come vengono salvati i dati nel DB in analisi;
- Differenze nella struttura delle query, ad esempio se è più o meno intuitiva rispetto ai linguaggi di interrogazione visti a lezione;
- Analisi sulla documentazione (e.g., più o meno intuitiva e semplice rispetto ai DB visti a lezione).

### Esempi di Query

Definire 5 query che ritenete rilevanti dopo aver osservato i dati inseriti all'interno del database e mostrare i risultati nelle celle in output. 

Almeno due delle 5 query devono risultare più complesse. La complessità che si prenderà in considerazione comprende queste caratteristiche:

- Query che aggregano dei dati;
- Query che contengono query innestate;
- Query che effettuano più operazioni.

### ADD A NEW TABLE OF ENZYMES

In [12]:
create_table_query = """
CREATE TABLE IF NOT EXISTS enzymes (
  "ActualKey" INT PRIMARY KEY,
  "DrugBank ID" TEXT,
  "Name" TEXT,
  "Type" TEXT,
  "UniProt ID" TEXT,
  "UniProt Name" TEXT
);
"""

insert_query = """
INSERT INTO enzymes ("ActualKey", "DrugBank ID", "Name", "Type", "UniProt ID", "UniProt Name")
VALUES (?, ?, ?, ?, ?, ?);
"""

data = load_json('enzymejsonfinal.json')
if data is None:
    print("Failed to load JSON data. Exiting.")
    exit(1)

try:
    cursor.execute(create_table_query)
    connection.commit()
except Exception as e:
    print(f"Error creating table: {e}")
    print(traceback.format_exc())

try:
    for record in data:

        cursor.execute(insert_query, (
            record['ActualKey'],
            record['DrugBank ID'],
            record['Name'],  
            record['Type'],
            record['UniProt ID'],
            record['UniProt Name'],
        ))
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())


### ADD A NEW TABLE OF CARRIERS

In [13]:
create_table_query = """
CREATE TABLE IF NOT EXISTS carriers (
  "DrugBank ID" TEXT,
  "Name" TEXT,
  "Type" TEXT,
  "UniProt ID" TEXT,
  "UniProt Name" TEXT,
  PRIMARY KEY ("DrugBank ID", "UniProt ID")
);
"""

insert_query = """
INSERT INTO carriers ("DrugBank ID", "Name", "Type", "UniProt ID", "UniProt Name")
VALUES (?, ?, ?, ?, ?);
"""

data = load_json('carrierjson.json')
if data is None:
    print("Failed to load JSON data. Exiting.")
    exit(1)

try:
    cursor.execute(create_table_query)
    connection.commit()
except Exception as e:
    print(f"Error creating table: {e}")
    print(traceback.format_exc())

try:
    for record in data:
        cursor.execute(insert_query, (
            record['DrugBank ID'],
            record['Name'],  
            record['Type'],
            record['UniProt ID'],
            record['UniProt Name'],
        ))
    connection.commit()
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())

In [14]:
# Query to list all tables, used to check that everything is ok
query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'doc';
"""

cursor.execute(query)
tables = cursor.fetchall()

for table in tables:
    print(table[0])


enzymes
carriers
drugs_external


### 1. Join: find ID and names of the drugs that are both enzymes and carriers

In [17]:
query = """
    SELECT ed."DrugBank ID", ed."Name"
    FROM drugs_external ed
    JOIN enzymes e ON ed."DrugBank ID" = e."DrugBank ID"
    JOIN carriers c ON ed."DrugBank ID" = c."DrugBank ID"
    GROUP BY ed."DrugBank ID", ed."Name";
"""

start_time = time.time()
try:
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute: {execution_time} seconds")

['DB00152', 'Thiamine']
['DB14002', 'D-alpha-Tocopherol acetate']
['DB00983', 'Formoterol']
['DB01582', 'Sulfamethazine']
['DB00526', 'Oxaliplatin']
['DB01384', 'Paramethasone']
['DB08931', 'Riociguat']
['DB09383', 'Meprednisone']
['DB01183', 'Naloxone']
['DB13944', 'Testosterone enanthate']
['DB14669', 'Betamethasone phosphate']
['DB09074', 'Olaparib']
['DB00624', 'Testosterone']
['DB01156', 'Bupropion']
['DB00861', 'Diflunisal']
['DB00934', 'Maprotiline']
['DB00501', 'Cimetidine']
['DB00682', 'Warfarin']
['DB01056', 'Tocainide']
['DB02709', 'Resveratrol']
['DB11751', 'Cabotegravir']
['DB00557', 'Hydroxyzine']
['DB00246', 'Ziprasidone']
['DB01120', 'Gliclazide']
['DB00575', 'Clonidine']
['DB00814', 'Meloxicam']
['DB00731', 'Nateglinide']
['DB00601', 'Linezolid']
['DB00758', 'Clopidogrel']
['DB00857', 'Terbinafine']
['DB06209', 'Prasugrel']
['DB01124', 'Tolbutamide']
['DB00316', 'Acetaminophen']
['DB00481', 'Raloxifene']
['DB00677', 'Isoflurophate']
['DB00737', 'Meclizine']
['DB01189',

### 2. Nested Query: find the names of drugs that have more than one carrier associated with them

In [19]:
nested_query = """
    SELECT ed."Name" AS drug_name
    FROM drugs_external ed
    WHERE ed."DrugBank ID" IN (
        SELECT c."DrugBank ID"
        FROM carriers c
        GROUP BY c."DrugBank ID"
        HAVING COUNT(c."UniProt ID") > 1
    );
    """

start_time = time.time()
try:
    cursor.execute(nested_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute: {execution_time} seconds")

['Myristic acid']
['Dolutegravir']
['Riociguat']
['Thyroid, porcine']
['Brexpiprazole']
['Oxygen']
['Omega-3-carboxylic acids']
['8-anilinonaphthalene-1-sulfonic acid']
['Liotrix']
['Resveratrol']
['Vitamin A']
['Amitriptyline']
['Ipratropium']
['Methadone']
['Alfuzosin']
['Clozapine']
['Spironolactone']
['Betamethasone']
['Buspirone']
['Oxycodone']
['Erlotinib']
['Diclofenac']
['Prednisone']
['Midazolam']
['Nateglinide']
['Estradiol']
['Alfentanil']
['Fentanyl']
['Temozolomide']
['Glycopyrronium']
['Clobetasol propionate']
['Irbesartan']
['Oxybutynin']
['Nifedipine']
['Mecasermin']
['Acenocoumarol']
['alpha-Tocopherol acetate']
['Hydrocortisone aceponate']
['Hydrocortisone butyrate']
['Ripretinib']
['Tepotinib']
['Calcium Phosphate']
['Vonoprazan']
['Infigratinib']
['Favipiravir']
['Lurbinectedin']
['Asenapine']
['Avanafil']
['Oxymetholone']
['Prednisolone acetate']
['Beta carotene']
['Ivacaftor']
['Crizotinib']
['Ulipristal']
['Mirabegron']
['Bedaquiline']
['Lacidipine']
['Ubidecaren

### 3. Query with multiple operations: finds the 5 drugs that are associated with the highest number of carriers

In [20]:
multiple_operations_query = """
    SELECT "DrugBank ID", "Name", carrier_count
    FROM (
        SELECT ed."DrugBank ID", ed."Name", COUNT(c."UniProt ID") AS carrier_count
        FROM drugs_external ed
        JOIN carriers c ON ed."DrugBank ID" = c."DrugBank ID"
        GROUP BY ed."DrugBank ID", ed."Name"
    ) sub
    ORDER BY carrier_count DESC
    LIMIT 5;
    """

start_time = time.time()
try:
    cursor.execute(multiple_operations_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute: {execution_time} seconds")

['DB09130', 'Copper', 25]
['DB00877', 'Sirolimus', 13]
['DB11886', 'Infigratinib', 11]
['DB00162', 'Vitamin A', 8]
['DB01277', 'Mecasermin', 7]
Time taken to execute: 0.03286457061767578 seconds


### 4. Complex Multi-Operation Query: provide an overview of the drugs that are associated with both enzymes and carriers, along with the number of enzymes and carriers, and the name and DrugBank ID of the drug, ordered by the number of associated enzymes and carriers in descending order

In [21]:
complex_multi_query = """
    SELECT ed."DrugBank ID", ed."Name", 
           COUNT(DISTINCT e."UniProt ID") AS enzyme_count,
           COUNT(DISTINCT c."UniProt ID") AS carrier_count
    FROM drugs_external ed
    LEFT JOIN enzymes e ON ed."DrugBank ID" = e."DrugBank ID"
    LEFT JOIN carriers c ON ed."DrugBank ID" = c."DrugBank ID"
    GROUP BY ed."DrugBank ID", ed."Name"
    HAVING COUNT(DISTINCT e."UniProt ID") > 0
       AND COUNT(DISTINCT c."UniProt ID") > 0
    ORDER BY enzyme_count DESC, carrier_count DESC;
    """

start_time = time.time()
try:
    cursor.execute(complex_multi_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute: {execution_time} seconds")

['DB12471', 'Ibrexafungerp', 24, 1]
['DB00675', 'Tamoxifen', 23, 2]
['DB00960', 'Pindolol', 23, 2]
['DB00783', 'Estradiol', 22, 3]
['DB00321', 'Amitriptyline', 22, 2]
['DB00714', 'Apomorphine', 21, 1]
['DB00252', 'Phenytoin', 20, 2]
['DB00503', 'Ritonavir', 20, 2]
['DB00997', 'Doxorubicin', 20, 1]
['DB00787', 'Acyclovir', 19, 1]
['DB12243', 'Edaravone', 19, 1]
['DB01026', 'Ketoconazole', 18, 2]
['DB13946', 'Testosterone undecanoate', 17, 2]
['DB00313', 'Valproic acid', 17, 1]
['DB13943', 'Testosterone cypionate', 16, 2]
['DB13944', 'Testosterone enanthate', 16, 2]
['DB00959', 'Methylprednisolone', 16, 1]
['DB00741', 'Hydrocortisone', 15, 2]
['DB00316', 'Acetaminophen', 15, 1]
['DB00515', 'Cisplatin', 15, 1]
['DB14649', 'Dexamethasone acetate', 15, 1]
['DB01234', 'Dexamethasone', 15, 1]
['DB00968', 'Methyldopa', 15, 1]
['DB00349', 'Clobazam', 14, 2]
['DB00624', 'Testosterone', 14, 2]
['DB00977', 'Ethinylestradiol', 14, 2]
['DB00586', 'Diclofenac', 14, 2]
['DB00818', 'Propofol', 14, 1]
[

### 5. Complex Nested Query: provide an overview of drugs associated with both enzymes and carriers that have an enzyme and carrier number greater than or equal to the average. Order the results by the number of associated enzymes and carriers in descending order

In [22]:
complex_nested_query = """
    SELECT
        outer_query."DrugBank ID",
        outer_query."Name",
        outer_query.enzyme_count,
        outer_query.carrier_count
    FROM (
        SELECT
            ed."DrugBank ID",
            ed."Name",
            COUNT(DISTINCT e."UniProt ID") AS enzyme_count,
            COUNT(DISTINCT c."UniProt ID") AS carrier_count
        FROM drugs_external ed
        JOIN enzymes e ON ed."DrugBank ID" = e."DrugBank ID"
        JOIN carriers c ON ed."DrugBank ID" = c."DrugBank ID"
        GROUP BY ed."DrugBank ID", ed."Name"
        HAVING COUNT(DISTINCT e."UniProt ID") > 0
           AND COUNT(DISTINCT c."UniProt ID") > 0
    ) AS outer_query
    WHERE outer_query.enzyme_count >= (
        SELECT AVG(inner_query.enzyme_count)
        FROM (
            SELECT
                COUNT(DISTINCT e."UniProt ID") AS enzyme_count
            FROM drugs_external ed
            JOIN enzymes e ON ed."DrugBank ID" = e."DrugBank ID"
            GROUP BY ed."DrugBank ID"
        ) AS inner_query
    )
    AND outer_query.carrier_count >= (
        SELECT AVG(inner_query.carrier_count)
        FROM (
            SELECT
                COUNT(DISTINCT c."UniProt ID") AS carrier_count
            FROM drugs_external ed
            JOIN carriers c ON ed."DrugBank ID" = c."DrugBank ID"
            GROUP BY ed."DrugBank ID"
        ) AS inner_query
    )
    ORDER BY outer_query.enzyme_count DESC, outer_query.carrier_count DESC;
    """

start_time = time.time()
try:
    cursor.execute(complex_nested_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing query: {e}")
    print(traceback.format_exc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute: {execution_time} seconds")

['DB00675', 'Tamoxifen', 23, 2]
['DB00960', 'Pindolol', 23, 2]
['DB00783', 'Estradiol', 22, 3]
['DB00321', 'Amitriptyline', 22, 2]
['DB00252', 'Phenytoin', 20, 2]
['DB00503', 'Ritonavir', 20, 2]
['DB01026', 'Ketoconazole', 18, 2]
['DB13946', 'Testosterone undecanoate', 17, 2]
['DB13944', 'Testosterone enanthate', 16, 2]
['DB13943', 'Testosterone cypionate', 16, 2]
['DB00741', 'Hydrocortisone', 15, 2]
['DB00624', 'Testosterone', 14, 2]
['DB00586', 'Diclofenac', 14, 2]
['DB00977', 'Ethinylestradiol', 14, 2]
['DB00349', 'Clobazam', 14, 2]
['DB01045', 'Rifampicin', 13, 3]
['DB00333', 'Methadone', 13, 2]
['DB14631', 'Prednisolone phosphate', 12, 2]
['DB00544', 'Fluorouracil', 12, 2]
['DB09130', 'Copper', 11, 25]
['DB11757', 'Istradefylline', 11, 3]
['DB00635', 'Prednisone', 11, 2]
['DB13955', 'Estradiol dienanthate', 10, 3]
['DB13952', 'Estradiol acetate', 10, 3]
['DB01115', 'Nifedipine', 10, 3]
['DB13954', 'Estradiol cypionate', 10, 3]
['DB13953', 'Estradiol benzoate', 10, 3]
['DB13956', '

## Testing the OBJECT(DYNAMIC) data type

### Create a sensor_data table with a OBJECT(dynamic) column

In [23]:
create_table_query = """
CREATE TABLE IF NOT EXISTS sensor_data (
    "sensor_id" STRING PRIMARY KEY,
    "timestamp" TIMESTAMP,
    "data" OBJECT(DYNAMIC)
)
"""

insert_query = """
INSERT INTO sensor_data ("sensor_id", "timestamp", "data")
VALUES (?, ?, ?);
"""

data = load_json('sensor_data.json')
if data is None:
    print("Failed to load JSON data. Exiting.")
    exit(1)

try:
    cursor.execute(create_table_query)
    connection.commit()
except Exception as e:
    print(f"Error creating table: {e}")
    print(traceback.format_exc())

try:
    for record in data:
        cursor.execute(insert_query, (
            record["sensor_id"],
            record["timestamp"],
            record["data"]
        ))
    connection.commit()
except Exception as e:
    print(f"Error inserting data: {e}")
    print(traceback.format_exc())


## Fetch all data

In [24]:
select_query = """
SELECT * FROM sensor_data;
"""

start_time = time.time()
try:
    cursor.execute(select_query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time taken to fetch: {execution_time} seconds")
except Exception as e:
    print(f"Error executing SELECT query: {e}")
    print(traceback.format_exc())

['sensor_7', 1718987007000, {'temperature': 33.7, 'humidity': 61.88, 'status': 'normal'}]
['sensor_16', 1718987007000, {'temperature': 31.11, 'humidity': 69.0, 'status': 'warning'}]
['sensor_17', 1718987007000, {'temperature': 19.41, 'humidity': 47.08, 'status': 'normal'}]
['sensor_23', 1718987007000, {'temperature': 21.2, 'humidity': 52.52, 'status': 'warning'}]
['sensor_24', 1718987007000, {'temperature': 22.55, 'humidity': 42.23, 'status': 'normal'}]
['sensor_31', 1718987007000, {'temperature': 16.13, 'humidity': 47.72, 'status': 'warning'}]
['sensor_47', 1718987007000, {'temperature': 32.04, 'humidity': 54.87, 'status': 'normal'}]
['sensor_59', 1718987007000, {'temperature': 23.93, 'humidity': 55.83, 'status': 'normal'}]
['sensor_67', 1718987007000, {'temperature': 30.93, 'humidity': 43.15, 'status': 'warning'}]
['sensor_75', 1718987007000, {'temperature': 27.68, 'humidity': 56.96, 'status': 'error'}]
['sensor_77', 1718987007000, {'temperature': 24.11, 'humidity': 55.49, 'status': 

## Calculate the average temperature from the data object for each sensor_id

In [25]:
query = """
    SELECT 
        sensor_id, 
        AVG(CAST(data['temperature'] AS DOUBLE)) AS avg_temperature 
    FROM 
        sensor_data 
    WHERE 
        data['temperature'] IS NOT NULL 
    GROUP BY 
        sensor_id;
"""

start_time = time.time()

try:
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(f"Sensor ID: {row[0]}, Average Temperature: {row[1]}")
except Exception as e:
    print(f"Error executing query: {e}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute query: {execution_time} seconds")

Sensor ID: sensor_4130, Average Temperature: 34.71
Sensor ID: sensor_3282, Average Temperature: 26.72
Sensor ID: sensor_8965, Average Temperature: 29.46
Sensor ID: sensor_9813, Average Temperature: 30.74
Sensor ID: sensor_1098, Average Temperature: 19.53
Sensor ID: sensor_4125, Average Temperature: 32.51
Sensor ID: sensor_8964, Average Temperature: 26.1
Sensor ID: sensor_9814, Average Temperature: 21.44
Sensor ID: sensor_4126, Average Temperature: 24.77
Sensor ID: sensor_5457, Average Temperature: 19.68
Sensor ID: sensor_8967, Average Temperature: 18.64
Sensor ID: sensor_8966, Average Temperature: 21.21
Sensor ID: sensor_5451, Average Temperature: 32.98
Sensor ID: sensor_6783, Average Temperature: 33.08
Sensor ID: sensor_3279, Average Temperature: 18.06
Sensor ID: sensor_8960, Average Temperature: 32.91
Sensor ID: sensor_9810, Average Temperature: 30.63
Sensor ID: sensor_4122, Average Temperature: 17.31
Sensor ID: sensor_5453, Average Temperature: 31.02
Sensor ID: sensor_7632, Average 

## Select all rows where the data['status'] is 'normal' and retrieves the sensor_id, timestamp, and data['status']

In [26]:
query = """
    SELECT 
        sensor_id, 
        timestamp, 
        data['status'] AS status 
    FROM 
        sensor_data 
    WHERE 
        data['status'] = 'normal';
"""

start_time = time.time()

try:
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(f"Sensor ID: {row[0]}, Timestamp: {row[1]}, Status: {row[2]}")

except Exception as e:
    print(f"Error executing query: {e}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Time taken to execute query: {execution_time} seconds")

Sensor ID: sensor_7, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_17, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_24, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_47, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_59, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_77, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_104, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_107, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_129, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_130, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_150, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_166, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_177, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_187, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_191, Timestamp: 1718987007000, Status: normal
Sensor ID: sensor_196, Timestamp: 1718987007000

In [27]:
# Query to drop every table, used to reset and run the Jupyter file with no problems
from sqlalchemy import create_engine, text

try:
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'doc';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    print("All tables dropped successfully.")
except Exception as e:
    print(f"Error dropping tables: {e}")
    print(traceback.format_exc())

All tables dropped successfully.


### Analisi delle tempistiche delle Query

Riportare le tempistiche delle query svolte nella sezione sopra. Per farlo è possibile procedere in con due modalità:

1. Attraverso un metodo (o un parametro) presente nel DBMS che restituisce il tempo di esecuzione;
2. Attraverso la libreria [time](https://docs.python.org/3/library/time.html) presente in Python per calcolare il lasso di tempo di esecuzione della query

Inserire un commento in merito ai tempi di esecuzione con eventuale indicazione dei metodi applicabili per il loro miglioramento.

### Contributors

I contributors, o contributori, ovvero gli studenti che hanno partecipato all'attività di progetto. Per ognuno di essi riportare **Nome**, **Cognome** e **Matricola**.

Silvia Cambiago 879382